论文中DAM图示如下：  
![](20240523103415.png)

上图网络结构如下：

1. **原始图像（记作1）**
2. **输入图像处理**：
    - **卷积层（绿色）**
    - **带泄漏的ReLU激活函数（黄色）**
    - 处理完成，记作2
3. **像素注意力（PA）模块**:
    - **卷积层（绿色）**
    - **带泄漏的ReLU激活函数（黄色）**
    - **批归一化层（蓝色）**
    - **卷积层（绿色）**
    - **Sigmoid函数（橙色）**
    - 处理完成，记作3.1
    - 3.1与2相乘，PA正式结束，记作3.2
4. **通道注意力（CA）模块**:
    - **全局平均池化（GAP，灰色）**
    - **卷积层（绿色）**
    - **带泄漏的ReLU激活函数（黄色）**
    - **卷积层（绿色）**
    - **Sigmoid函数（橙色）**
    - 处理完成，记作3.3
    - 3.3与2相乘，记作3.4
    - CA处理完成
5. **融合操作**:
    - 3.2与3.4连接，记作4
6. **处理融合结果**:
    - **卷积层（绿色）**
    - **Tanh激活函数（浅粉色）**
    - 处理完成，记作5
7. **输出结果**:
    - 5和两个1以及2相加，处理完成


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DualAttentionModule(nn.Module):
    def __init__(self, in_channels):
        super(DualAttentionModule, self).__init__()

        # 输入图像的初始处理
        self.initial_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.initial_relu = nn.LeakyReLU(0.1)
        
        # 像素注意力模块
        self.pa_conv1 = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.pa_relu1 = nn.LeakyReLU(0.1)
        self.pa_bn1 = nn.BatchNorm2d(in_channels // 8)
        self.pa_conv2 = nn.Conv2d(in_channels // 8, 1, kernel_size=1)
        self.pa_sigmoid = nn.Sigmoid()

        # 通道注意力模块
        self.ca_gap = nn.AdaptiveAvgPool2d(1)
        self.ca_conv1 = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.ca_relu1 = nn.LeakyReLU(0.1)
        self.ca_conv2 = nn.Conv2d(in_channels // 8, in_channels, kernel_size=1)
        self.ca_sigmoid = nn.Sigmoid()

        # 特征增强模块
        self.fe_conv = nn.Conv2d(in_channels * 2, in_channels, kernel_size=3, padding=1)
        self.fe_tanh = nn.Tanh()

    def forward(self, x):
        # 初始处理
        x2 = self.initial_conv(x)
        x2 = self.initial_relu(x2)
        
        # 像素注意力CA
        pa = self.pa_conv1(x2)
        pa = self.pa_relu1(pa)
        pa = self.pa_bn1(pa)
        pa = self.pa_conv2(pa)
        pa = self.pa_sigmoid(pa)
        pa_out = x2 * pa
        
        # 通道注意力PA
        ca = self.ca_gap(x2)
        ca = self.ca_conv1(ca)
        ca = self.ca_relu1(ca)
        ca = self.ca_conv2(ca)
        ca = self.ca_sigmoid(ca)
        ca_out = x2 * ca
        
        # 拼接PA和CA的输出
        concat = torch.cat([pa_out, ca_out], dim=1)
        
        # 最后两层
        fe = self.fe_conv(concat)
        fe = self.fe_tanh(fe)
        
        # 输出
        out = fe + x + x + x2
        
        return out

class HDRNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HDRNet, self).__init__()
        self.dam1 = DualAttentionModule(in_channels)
        self.dam2 = DualAttentionModule(in_channels)
        self.final_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.dam1(x)
        x = self.dam2(x)
        x = self.final_conv(x)
        return x

# 测试
model = HDRNet(in_channels=64, out_channels=64)
input_image = torch.randn(1, 64, 256, 256)  # 输入张量
output_image = model(input_image)
print(output_image.shape)
